In [ ]:
!pip install yt-dlp transformers gradio moviepy gtts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB

openai/whisper-small

In [27]:
from transformers import pipeline
import gradio as gr
from gtts import gTTS
import yt_dlp


pipe = pipeline(model="momowanwu/checkpoint2")

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text
def transcribe(audio_file_path):
    try:
        print("Received audio file path:", audio_file_path)

        text = pipe(audio_file_path)["text"]
        return text
    except Exception as e:
        print("Error during transcription:", e)
        return f"An error occurred: {e}"

def transcribe_from_url(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '/tmp/%(extractor)s-%(id)s-%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'noplaylist': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            audio_file = ydl.prepare_filename(info_dict)
            audio_file = audio_file.rsplit(".", 1)[0] + ".wav"
            text = pipe(audio_file)["text"]
            return text
    except Exception as e:
        return f"An error occurred: {e}"


def synthesize_speech(text):
    tts = gTTS(text=text, lang='zh-cn')
    audio_file = 'synthesized_speech.mp3'
    tts.save(audio_file)
    return audio_file

with gr.Blocks() as demo:
    gr.Markdown("### Whisper Small Chinese")
    gr.Markdown("Realtime demo for Chinese speech recognition and text to speech synthesis using a fine-tuned Whisper small model.")


    with gr.Tab("Transcribe from URL"):
        url_input = gr.Textbox(label="Enter URL of an audio file")
        url_transcribe_button = gr.Button("Transcribe from URL")
        url_transcription_output = gr.Textbox(label="Transcription from URL")
        url_input.change(transcribe_from_url, inputs=url_input, outputs=url_transcription_output)

    with gr.Tab("Transcribe"):
        with gr.Column():
            audio_input = gr.Audio(label="Record your speech or upload an audio file", type="filepath")
            transcription_output = gr.Textbox(label="Transcription")
            audio_input.change(transcribe, inputs=audio_input, outputs=transcription_output)

    with gr.Tab("Text to Speech Synthesis"):
        text_input = gr.Textbox(label="Enter text to synthesize")
        synthesize_button = gr.Button("Synthesize")
        synthesized_speech_output = gr.Audio(label="Synthesized Speech")
        text_input.change(synthesize_speech, inputs=text_input, outputs=synthesized_speech_output)

demo.launch()



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f02f5a1d67c71070ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
